
---

# 💾 **PVC · Model Storage · Cache**

---

## 🔑 What “state” means (LLM/Agents)

* **📦 Models** → weights, tokenizer
* **⚡ Caches** → HuggingFace cache, compiled artifacts
* **🗄️ Data stores** → vector DB, Postgres
* **📨 Queues** → Redis, Kafka

---

## 📂 Storage patterns

1. **🗄️ RWX PVC (best)** → shared, read-only `/models`.
2. **📥 RWO + initContainer** → pod downloads model at start.
3. **📦 Warm image** → bake weights into image (fast, heavy).
   👉 Always persist **HF cache** on PVC to skip re-downloads.

---

## 📝 PVC examples

**RWX (shared):**

```yaml
accessModes: [ReadWriteMany]
resources: { requests: { storage: 200Gi } }
```

**RWO (per pod):**

```yaml
accessModes: [ReadWriteOnce]
resources: { requests: { storage: 200Gi } }
```

---

## 🔗 Mounts (pods)

```yaml
volumeMounts:
  - { name: model,   mountPath: /models, readOnly: true }
  - { name: hfcache, mountPath: /home/app/.cache/huggingface }
```

---

## 🚀 InitContainer (download model)

```yaml
initContainers:
  - name: fetch
    image: ghcr.io/huggingface/cli:latest
    args: ["huggingface-cli","download","meta-llama/Llama-3-8b-instruct","--local-dir","/models/llama"]
    volumeMounts: [{ name: model, mountPath: /models }]
```

---

## 🔒 Permissions

```yaml
securityContext:
  runAsNonRoot: true
  fsGroup: 1000
```

---

## ⚡ Faster starts

* Pre-populate PVC once with a **Job**.
* Cache models on PVC (or **emptyDir** for temp SSD speed).
* Always mount `/models:ro`.

---

## ✅ Checklist

* [ ] RWX or RWO+init chosen
* [ ] `/models:ro` + hf-cache PVC mounted
* [ ] `fsGroup` for non-root access
* [ ] Tokens in **Secret**
* [ ] Optionally pre-warm models with a **Job**

---
